In [ ]:
!pip install causalinference

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from causalinference import CausalModel
import warnings
warnings.filterwarnings('ignore')

In [3]:
datasets = pd.read_csv('/content/drive/MyDrive/CausalInference/sales_dataset.csv')
datasets.head()

,order_id,order_date,status,item_id,sku,qty_ordered,price,value,discount_amount,total,...,SSN,Phone No.,Place Name,County,City,State,Zip,Region,User Name,Discount_Percent
0,100354678,2020-10-01,received,574772.0,oasis_Oasis-064-36,22.0,89.9,1798.0,0.0,1798.0,...,627-31-5251,405-959-1129,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0
1,100354678,2020-10-01,received,574774.0,Fantastic_FT-48,12.0,19.0,190.0,0.0,190.0,...,627-31-5251,405-959-1129,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0
2,100354680,2020-10-01,complete,574777.0,mdeal_DMC-610-8,10.0,149.9,1199.2,0.0,1199.2,...,627-31-5251,405-959-1129,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0
3,100354680,2020-10-01,complete,574779.0,oasis_Oasis-061-36,10.0,79.9,639.2,0.0,639.2,...,627-31-5251,405-959-1129,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0
4,100367357,2020-11-13,received,595185.0,MEFNAR59C38B6CA08CD,3.0,99.9,99.9,0.0,99.9,...,627-31-5251,405-959-1129,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0


In [4]:
datasets.shape

(286392, 36)

In [5]:
datasets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286392 entries, 0 to 286391
Data columns (total 36 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   order_id          286392 non-null  object 
 1   order_date        286392 non-null  object 
 2   status            286392 non-null  object 
 3   item_id           286392 non-null  float64
 4   sku               286392 non-null  object 
 5   qty_ordered       286392 non-null  float64
 6   price             286392 non-null  float64
 7   value             286392 non-null  float64
 8   discount_amount   286392 non-null  float64
 9   total             286392 non-null  float64
 10  category          286392 non-null  object 
 11  payment_method    286392 non-null  object 
 12  bi_st             286392 non-null  object 
 13  cust_id           286392 non-null  float64
 14  year              286392 non-null  int64  
 15  month             286392 non-null  object 
 16  ref_num           28

In [6]:
# Check for the treatment values

datasets['Discount_Percent'].value_counts()

0.000000     183086
20.000000     10510
10.000000      8459
15.000000      4985
9.000000       2074
              ...  
17.016538         1
17.128112         1
3.671476          1
12.241249         1
62.727273         1
Name: Discount_Percent, Length: 17133, dtype: int64

In [7]:
# Check for distribution of region

datasets['Region'].value_counts()

South        103482
Midwest       81299
West          51080
Northeast     50531
Name: Region, dtype: int64

In [8]:
# Check for the category of products sold

datasets['category'].value_counts()

Mobiles & Tablets     61761
Men's Fashion         40713
Appliances            33034
Women's Fashion       28334
Others                26108
Beauty & Grooming     17899
Entertainment         17352
Superstore            15024
Home & Living         13990
Health & Sports        8421
Computing              8110
Soghaat                7250
Kids & Baby            6492
School & Education     1090
Books                   814
Name: category, dtype: int64

In [9]:
# Check status of the customers

datasets['Name Prefix'].value_counts()

Mr.      103506
Ms.       60314
Mrs.      47247
Hon.      30439
Drs.      16180
Prof.     14939
Dr.       13767
Name: Name Prefix, dtype: int64

In [10]:
# Feature Selection

datasets.columns

Index(['order_id', 'order_date', 'status', 'item_id', 'sku', 'qty_ordered',
       'price', 'value', 'discount_amount', 'total', 'category',
       'payment_method', 'bi_st', 'cust_id', 'year', 'month', 'ref_num',
       'Name Prefix', 'First Name', 'Middle Initial', 'Last Name', 'Gender',
       'age', 'full_name', 'E Mail', 'Customer Since', 'SSN', 'Phone No. ',
       'Place Name', 'County', 'City', 'State', 'Zip', 'Region', 'User Name',
       'Discount_Percent'],
      dtype='object')

In [11]:
variables = ['order_date','qty_ordered','price', 'value', 'discount_amount', 'total', 'category','payment_method',
'Name Prefix','Gender','age','Customer Since','Region','Discount_Percent']

In [12]:
data = datasets[variables]
data.head()

,order_date,qty_ordered,price,value,discount_amount,total,category,payment_method,Name Prefix,Gender,age,Customer Since,Region,Discount_Percent
0,2020-10-01,22.0,89.9,1798.0,0.0,1798.0,Men's Fashion,cod,Drs.,F,43.0,8/22/2006,South,0.0
1,2020-10-01,12.0,19.0,190.0,0.0,190.0,Men's Fashion,cod,Drs.,F,43.0,8/22/2006,South,0.0
2,2020-10-01,10.0,149.9,1199.2,0.0,1199.2,Men's Fashion,cod,Drs.,F,43.0,8/22/2006,South,0.0
3,2020-10-01,10.0,79.9,639.2,0.0,639.2,Men's Fashion,cod,Drs.,F,43.0,8/22/2006,South,0.0
4,2020-11-13,3.0,99.9,99.9,0.0,99.9,Men's Fashion,cod,Drs.,F,43.0,8/22/2006,South,0.0


In [13]:
# Extract data for women's fashion

w_data = data[data['category']=="Women's Fashion"]

In [14]:
# Calculate how long customer has been with the company

import datetime as dt

In [15]:
# convert the dates from string to dates

w_data['order_date'] = pd.to_datetime(w_data['order_date'])
w_data['customer_since'] = pd.to_datetime(w_data['Customer Since'])

In [16]:
w_data['diff'] = w_data['order_date'] - w_data['customer_since']

In [17]:
w_data['diff'] = w_data['diff'].dt.days

In [18]:
w_data.head()

,order_date,qty_ordered,price,value,discount_amount,total,category,payment_method,Name Prefix,Gender,age,Customer Since,Region,Discount_Percent,customer_since,diff
29,2020-10-01,3.0,140.0,140.0,0.0,140.0,Women's Fashion,cod,Mr.,M,65.0,6/27/2010,Midwest,0.0,2010-06-27,3749
72,2020-12-08,3.0,379.8,379.8,0.0,379.8,Women's Fashion,Payaxis,Ms.,F,71.0,3/31/2017,Midwest,0.0,2017-03-31,1348
158,2020-10-01,3.0,175.0,175.0,0.0,175.0,Women's Fashion,cod,Mr.,M,71.0,3/11/1986,South,0.0,1986-03-11,12623
165,2020-12-13,3.0,119.9,119.9,0.0,119.9,Women's Fashion,cod,Mr.,M,64.0,12/22/2005,South,0.0,2005-12-22,5470
166,2020-12-20,3.0,140.0,140.0,14.0,126.0,Women's Fashion,Easypay_MA,Mr.,M,64.0,12/22/2005,South,10.0,2005-12-22,5477


In [19]:
# Convert diff to years

w_data['cus_duration'] = round(w_data['diff']/365,0)

In [20]:
w_data.head()

,order_date,qty_ordered,price,value,discount_amount,total,category,payment_method,Name Prefix,Gender,age,Customer Since,Region,Discount_Percent,customer_since,diff,cus_duration
29,2020-10-01,3.0,140.0,140.0,0.0,140.0,Women's Fashion,cod,Mr.,M,65.0,6/27/2010,Midwest,0.0,2010-06-27,3749,10.0
72,2020-12-08,3.0,379.8,379.8,0.0,379.8,Women's Fashion,Payaxis,Ms.,F,71.0,3/31/2017,Midwest,0.0,2017-03-31,1348,4.0
158,2020-10-01,3.0,175.0,175.0,0.0,175.0,Women's Fashion,cod,Mr.,M,71.0,3/11/1986,South,0.0,1986-03-11,12623,35.0
165,2020-12-13,3.0,119.9,119.9,0.0,119.9,Women's Fashion,cod,Mr.,M,64.0,12/22/2005,South,0.0,2005-12-22,5470,15.0
166,2020-12-20,3.0,140.0,140.0,14.0,126.0,Women's Fashion,Easypay_MA,Mr.,M,64.0,12/22/2005,South,10.0,2005-12-22,5477,15.0


In [21]:
w_data['Month'] = w_data['order_date'].dt.strftime('%B')

In [22]:
w_data['Month'].value_counts()

December     7514
June         2744
January      2433
May          2239
April        2232
March        2104
November     2059
July         1590
September    1455
February     1412
October      1381
August       1171
Name: Month, dtype: int64

In [23]:
# Create 4 seasons

def season(x):
    if x in ['March','April','May']:
        return 'Spring'
    elif x in ['June','July','August']:
        return 'Summer'
    elif x in ['September','October','November']:
        return 'Fall'
    else:
        return 'Winter'

In [24]:
w_data['season'] = w_data['Month'].apply(season)

In [25]:
#  Qualifications

def status(x):
    if x in ['Drs.','Dr.','Prof.']:
        return 'Doctorate_Degree'
    else:
        return 'Others'

In [26]:
w_data['degree'] = w_data['Name Prefix'].apply(status)

In [27]:
# Rename total as Sales, and customer_duration as duration in both groups

new_names = {'total':'outcome','Discount_Percentage':'discount','payment_method':'paymethod'}
w_data.rename(columns=new_names,inplace=True)

In [28]:
# Create bins for customers age

def catage(x):
    if x <= 20:
        return '< 20 yrs'
    elif x <= 30:
        return '21-30 yrs'
    elif x <= 40:
        return '31-40 yrs'
    elif x <= 45:
        return '41-45 yrs'
    elif x <= 50:
        return '46-50 yrs'
    elif x <= 60:
        return '51-60 yrs'
    else:
        return 'above 60 yrs'

In [29]:
w_data['agecat'] = w_data['age'].apply(catage)

In [30]:
### Select the covariates for Propensity Score Matching

w_data.columns

Index(['order_date', 'qty_ordered', 'price', 'value', 'discount_amount',
       'outcome', 'category', 'paymethod', 'Name Prefix', 'Gender', 'age',
       'Customer Since', 'Region', 'Discount_Percent', 'customer_since',
       'diff', 'cus_duration', 'Month', 'season', 'degree', 'agecat'],
      dtype='object')

In [31]:
covariates = ['price','outcome', 'Gender','Region', 'Discount_Percent', 'cus_duration',
              'season', 'degree', 'agecat']

In [32]:
### Select control and treatment groups

df = w_data[covariates]
treatment_pop = df[df['Discount_Percent']>0.00]
control_pop = df[df['Discount_Percent']==0.00]

In [33]:
treatment_pop.shape,control_pop.shape

((6309, 9), (22025, 9))

In [34]:
treatment_pop['treatment'] =1
control_pop['treatment']=0

In [35]:
### Select a random sample of 1200 treatment and 1500 untreated

treatment_sample = treatment_pop.sample(n=1200,random_state=42)
control_sample = control_pop.sample(n=1600,random_state=42)

In [36]:
# combine the datasets

df_w = pd.concat([treatment_sample,control_sample],axis=0)

In [37]:
df_w.shape

(2800, 10)

In [38]:
df_w.head()

,price,outcome,Gender,Region,Discount_Percent,cus_duration,season,degree,agecat,treatment
71938,69.90,62.910,M,Northeast,10.000000,19.0,Winter,Others,31-40 yrs,1
153603,74.25,59.896,F,South,19.331987,22.0,Winter,Others,21-30 yrs,1
10237,69.90,62.910,F,South,10.000000,6.0,Winter,Others,46-50 yrs,1
49789,265.00,0.000,F,Midwest,9.433962,5.0,Fall,Others,51-60 yrs,1
251408,259.00,543.900,F,Northeast,30.000000,9.0,Summer,Others,41-45 yrs,1


In [39]:
df_w.tail()

,price,outcome,Gender,Region,Discount_Percent,cus_duration,season,degree,agecat,treatment
166483,49.90,49.9,F,Northeast,0.0,17.0,Winter,Doctorate_Degree,31-40 yrs,0
168442,447.50,447.5,M,South,0.0,5.0,Winter,Others,46-50 yrs,0
110335,201.75,1614.0,M,Northeast,0.0,31.0,Fall,Others,21-30 yrs,0
119697,149.90,149.9,F,South,0.0,22.0,Winter,Others,51-60 yrs,0
241347,99.50,99.5,F,South,0.0,19.0,Spring,Others,41-45 yrs,0


In [40]:
# Create causalinference variables

X = df_w.drop(['Discount_Percent','treatment','outcome'],axis=1)
X = pd.get_dummies(X,drop_first=True,prefix_sep=('*'))
X = X.values
D = df_w['treatment'].values
Y = df_w['outcome'].values

In [41]:
# initiate causal model and get the summary statistics of the dataset

causal = CausalModel(Y,D,X)

In [42]:
print(causal.summary_stats)


Summary Statistics

                      Controls (N_c=1600)        Treated (N_t=1200)             
       Variable         Mean         S.d.         Mean         S.d.     Raw-diff
--------------------------------------------------------------------------------
              Y      231.340      396.626      248.989      260.783       17.648

                      Controls (N_c=1600)        Treated (N_t=1200)             
       Variable         Mean         S.d.         Mean         S.d.     Nor-diff
--------------------------------------------------------------------------------
             X0      153.180      193.285      190.276      184.288        0.196
             X1       13.006        8.411       13.311        8.509        0.036
             X2        0.502        0.500        0.512        0.500        0.020
             X3        0.186        0.389        0.237        0.425        0.124
             X4        0.421        0.494        0.347        0.476       -0.151
      

In [43]:
causal.est_propensity_s()
print(causal.propensity)


Estimated Parameters of Propensity Score

                    Coef.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
     Intercept     -0.324      0.157     -2.070      0.038     -0.631     -0.017
            X0      0.001      0.001      1.373      0.170     -0.000      0.002
            X6     -1.068      0.170     -6.262      0.000     -1.402     -0.733
            X4     -0.245      0.113     -2.156      0.031     -0.467     -0.022
            X7     -0.715      0.150     -4.761      0.000     -1.009     -0.421
            X3      0.062      0.158      0.395      0.693     -0.248      0.373
            X5      0.931      0.235      3.959      0.000      0.470      1.392
           X15     -0.279      0.155     -1.797      0.072     -0.582      0.025
            X1     -0.005      0.007     -0.640      0.522     -0.019      0.010
         X0*X0     -0.000      0.000     -4.891      0.000     -0.

In [44]:
causal.trim_s()
causal.cutoff

0.13183943907910128

In [45]:
print(causal.summary_stats)


Summary Statistics

                      Controls (N_c=1597)        Treated (N_t=1195)             
       Variable         Mean         S.d.         Mean         S.d.     Raw-diff
--------------------------------------------------------------------------------
              Y      227.711      381.034      246.506      256.577       18.795

                      Controls (N_c=1597)        Treated (N_t=1195)             
       Variable         Mean         S.d.         Mean         S.d.     Nor-diff
--------------------------------------------------------------------------------
             X0      149.405      156.297      187.120      177.717        0.225
             X1       13.003        8.398       13.279        8.498        0.033
             X2        0.502        0.500        0.512        0.500        0.021
             X3        0.185        0.389        0.237        0.425        0.126
             X4        0.421        0.494        0.346        0.476       -0.153
      

In [46]:
causal.stratify_s()
print(causal.strata)


Stratification Summary

              Propensity Score         Sample Size     Ave. Propensity   Outcome
   Stratum      Min.      Max.  Controls   Treated  Controls   Treated  Raw-diff
--------------------------------------------------------------------------------
         1     0.156     0.282       281        70     0.237     0.242   -72.306
         2     0.282     0.364       237       112     0.326     0.330   -31.583
         3     0.364     0.428       424       275     0.396     0.396   -43.665
         4     0.428     0.501       334       362     0.461     0.462    13.839
         5     0.501     0.527        98        78     0.515     0.515   -20.458
         6     0.527     0.552        71       102     0.539     0.542    63.555
         7     0.552     0.596        85        89     0.573     0.572    91.872
         8     0.596     0.645        40        47     0.617     0.620    51.287
         9     0.646     0.868        27        60     0.701     0.717  -152.190



In [48]:
causal.est_via_ols()
causal.est_via_weighting()
causal.est_via_matching(bias_adj=True)
print(causal.estimates)


Treatment Effect Estimates: OLS

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE    -15.380      9.523     -1.615      0.106    -34.046      3.286
           ATC    -19.163      9.096     -2.107      0.035    -36.991     -1.336
           ATT    -10.325     10.716     -0.963      0.335    -31.328     10.679

Treatment Effect Estimates: Weighting

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE    -27.859     14.013     -1.988      0.047    -55.325     -0.393

Treatment Effect Estimates: Matching

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE    -34.622     17.613     -1.966      0.049    -69.144     -0.